<a href="https://colab.research.google.com/github/diyanigam/CookBook/blob/main/finetune_LLAMA2_(github_copy).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers==4.31.0
!pip install -U bitsandbytes
!pip install accelerate
!pip install peft==0.4.0 trl==0.4.7

In [ ]:
import os
import torch
from datasets import load_dataset
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
)
from peft import LoraConfig, PeftModel
from trl import SFTTrainer

In [ ]:
import json
data=json.load(open("/content/recipes.json",'r')) #recipes webscraped dataset

In [ ]:
new_data=[]
for x in data:
  temp="<s>[INST]"+x['Name']+"[/INST]"
  for y in x:
    if y!="Name":
      temp+=str(x[y])
  temp+="</s>"
  new_data.append(temp)

In [ ]:
temp

In [ ]:
new_data={"text":new_data}

In [ ]:
output_dir = "./results"

In [ ]:
import pandas as pd
pd.DataFrame(new_data).to_csv("/content/new_data.csv")

In [ ]:
from datasets import load_dataset
new_data=load_dataset("csv",data_files="/content/new_data.csv",split="train")

In [ ]:
new_data

In [ ]:
dataset = new_data

compute_dtype = getattr(torch, "bfloat16")

bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = "nf4",
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant = False,
)
# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and True:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# Load base model
access_token = "*********"
model_id = "meta-llama/Llama-2-7b-chat-hf"

tokenizer = AutoTokenizer.from_pretrained(model_id, use_auth_token=access_token, timeout=60)

# Load model with 4-bit quantization and float16 compute dtype
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    load_in_4bit=True,
    quantization_config=BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_use_double_quant=False,
    )
    use_auth_token=access_token,
    device_map="auto",  # Let transformers handle device placement
    trust_remote_code=True
)

model.config.use_cache = False
model.config.pretraining_tp = 1

# Load LLaMA tokenizer
tokenizer = AutoTokenizer.from_pretrained( "meta-llama/Llama-2-7b-chat-hf", trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right" # Fix weird overflow issue with fp16 training

# Load LoRA configuration
peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

# Set training parameters
training_arguments = TrainingArguments(
    output_dir=output_dir,
    num_train_epochs=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=20,
    optim="paged_adamw_32bit",
    save_steps=0,
    logging_steps=25,
    learning_rate=2e-4,
    weight_decay=0.001,
    fp16=False,
    bf16=True,
    max_grad_norm=0.3,
    max_steps=-1,
    warmup_ratio=0.03,
    group_by_length=True,
    lr_scheduler_type="cosine",
    report_to="tensorboard",
    gradient_checkpointing=True
)

# Set supervised fine-tuning parameters
trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=peft_config,
    dataset_text_field="text",
    #max_seq_length=None,
    tokenizer=tokenizer,
    args=training_arguments,
    packing=False,
    max_seq_length=256,
)



In [ ]:
model.train()

In [ ]:
from peft import get_peft_model

model = get_peft_model(model, peft_config)
model.print_trainable_parameters()


In [ ]:
for name, param in model.named_parameters():
    if "lora" in name:  # Check LoRA layers
        print(f"{name}: requires_grad={param.requires_grad}")

In [ ]:
torch.compile(model)

In [ ]:
# Train model
trainer.train()

In [ ]:
trainer.model.save_pretrained("cookbook_model")

In [ ]:
!pip install -q transformers datasets huggingface_hub

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
from huggingface_hub import HfApi

repo_id = "diyanigam/CookBook"

# Upload model
trainer.model.push_to_hub(repo_id)

In [ ]:
from peft import PeftModel

base_model = "meta-llama/Llama-2-7b-hf"
model = AutoModelForCausalLM.from_pretrained(base_model)
lora_model = PeftModel.from_pretrained(model, "cookbook_model")

lora_model.push_to_hub(repo_id)